In [1]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# fix professors
from owlready2 import *
main_onto_name = 'main-ResearchMentorOntology.owl'
onto_path.append("/Users/lauramiron/Desktop/CS270/project")
onto = get_ontology(main_onto_name)
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
onto.load()

for i in vivoNS.FacultyMember.instances():
    for RA in i.hasResearchArea:
        i.hasResearchArea.remove(RA)
onto.save()

In [7]:
import pdb
med_students_base_url = 'https://med.stanford.edu/profiles/browse?p={0}&affiliations=capMdStudent&ps=100'

driver = webdriver.Chrome()
med_student_dict = {}
print('scraping med students...')
for pg_num in range(1,4):
    print('downloading page '+str(pg_num))
    url = med_students_base_url.format(pg_num)
    driver.get(url)
    time.sleep(8)
    profiles = driver.find_elements_by_class_name("mini-profile")
    for pfl in profiles:
        media_heading_elem = pfl.find_element_by_class_name("media-heading")
        url_name = media_heading_elem.get_attribute("href").split('/')[-1]
        name = media_heading_elem.find_element_by_tag_name('h4').get_attribute('innerText')
        first_name = name.split(' ')[0].replace(' ','')
        last_name = name.split(' ')[-1].replace(' ','')
        program_info = media_heading_elem.find_element_by_tag_name('h5').get_attribute('innerText')
        program = program_info.split(' ')[0]
        grad_year = program_info.split(' ')[-1]
        med_student_dict[url_name] = {'firstName':first_name, 'lastName': last_name, 'program': program, 'gradDate': grad_year}
driver.quit()
med_student_dict_name = 'med_student_simple.p'
print('saving to '+med_student_dict_name)        
with open(med_student_dict_name,'wb+') as f:
    pickle.dump(med_student_dict,f)

scraping med students...
downloading page 1
downloading page 2
downloading page 3
saving to med_student_simple.p


In [1]:
from owlready2 import *

In [14]:
med_student_profile_url = 'https://med.stanford.edu/profiles/'
main_onto_name = 'main-ResearchMentorOntology.owl'
med_student_dict_name = 'med_student_simple.p'
med_student_dict = pickle.load(open(med_student_dict_name,'rb'))

onto_path.append("/Users/lauramiron/Desktop/CS270/project")
onto = get_ontology(main_onto_name)
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
onto.load()

for url_name, values in med_student_dict.items():
    first_name = values['firstName']
    last_name = values['lastName']
    program = values['program']
    grad_year = values['gradDate']
    
    med_indiv = onto.MedicalStudent((first_name+last_name).replace(' ',''))
    med_indiv.firstName = first_name
    med_indiv.lastName = last_name
    med_indiv.graduationDateExp.append(grad_year)
    med_indiv.uri = url_name
    med_indiv.hasProfilePage.append(med_student_profile_url+url_name)

onto.save()

In [ ]:
# add MESH terms and PubMed articles
